In [4]:
# import library
!pip install nltk
import nltk
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import re

# download APIs for natural language processing
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk

True

In [5]:
import warnings as wrn
wrn.filterwarnings('ignore')
# read data from CSV file
df = pd.read_csv('../data/raw/MarvelTheFantasticFourFirstSteps.csv')
# df = pd.read_csv('marvelRanarok.csv', encoding="cp1252")

# print(df["sentiment"].value_counts())
review = df["review"].tolist()

# convert label from text to digit process
positives = df[df["sentiment"] == "positive"] # group all the positive comments
negatives = df[df["sentiment"] == "negative"] # group all the negative comments
neutrals = df[df["sentiment"] == "neutral"]   # group all the neutral comments
negatives["sentiment"] = -1 # convert text label to digit lable -1 for negative
positives["sentiment"] = 1  # convert text label to digit lable 1 for positive
neutrals["sentiment"] = 0   # convert text label to digit lable 0 for neutral


# df["sentiment"] = df["sentiment"].map({"positive" : 1, "negative" : -1, "neutral" : 0})
# sentiment = df["sentiment"].tolist()
# Combine pos, neg, neutral data into one DataFrame data using pd.concat()
data = pd.concat([positives,
                  neutrals,
                  negatives,
                 ],axis=0)

data.reset_index(inplace=True) # reset the index of each row

print(data["sentiment"].value_counts())
data.head()
# data.info()
# print(review)
# print(sentiment)
# print(df.to_string())
# print(df.loc[2, "review"])
# print(df.info())
# df.head()
# pd.options.display.max_rows = 9999
# print(pd.options.display.max_rows)
# csv_size = len(df)


sentiment
-1    70
 1    50
 0    21
Name: count, dtype: int64


,index,review,sentiment
0,0,The saddest thing about this new movie is the ...,1
1,6,"Amen to that!!! Marvel, I love you forever 300...",1
2,8,Dude this trailer gives me goosebumps,1
3,10,Just saw this movie earlier today and omg it w...,1
4,11,I love how it’s a running joke that Ben simply...,1


In [6]:
# create pipeline for processing data
# import word tokenizer for tokenization process and pos_tag for speech tagging 
from nltk.tokenize import word_tokenize
from nltk import pos_tag

def preprocess_data(review):
    # Cleaning links
    text = re.sub(r'http\S+', '', review)
    
    # Cleaning everything except alphabetical and numerical characters
    text = re.sub("[^a-zA-Z0-9]"," ",review)
    
    # Tokenization
    token = word_tokenize(text)
    # tagged = pos_tag(token)
    # keep adjactive (JJ), adverb (RB) and verb (VB)
    # important_tokens = [w for w, pos in tagged if pos.startswith("JJ") or pos.startswith("RB") or pos.startswith("VB") or pos.startswith("NN")]
    preprocessed_text = " ".join(token)
    return preprocessed_text

preprocessed_text = [preprocess_data(text) for text in data["review"]]
print(preprocessed_text)


['The saddest thing about this new movie is the fact that Stan Lee and Jack Kirby bless their hearts are no longer around to see one of their most important creations after years of mistreatment finally done justice on the big screen', 'Amen to that Marvel I love you forever 3000', 'Dude this trailer gives me goosebumps', 'Just saw this movie earlier today and omg it was so good', 'I love how it s a running joke that Ben simply refuses to say ITS CLOBBERIN TIME Can t wait to see this in IMAX', 'One of the small things I love most about this movie was that Ben wasn t ever at one point ever called the thing It was truly about family Dom would be proud', 'It s gon na feel like ascending when Ben says It s Clobbering Time', 'I JUST SAW IT ON IMAX it was Fantastic', 'Saw this last night and I have to say I absolutely loved it Where the new superman movie has heart this has soul', 'I like the idea of Franklin Richards sneezing basically resetting the entire MCU since he can warp change reali

In [7]:
for i in range(0,5):
    print(preprocessed_text[i],end="\n")

The saddest thing about this new movie is the fact that Stan Lee and Jack Kirby bless their hearts are no longer around to see one of their most important creations after years of mistreatment finally done justice on the big screen
Amen to that Marvel I love you forever 3000
Dude this trailer gives me goosebumps
Just saw this movie earlier today and omg it was so good
I love how it s a running joke that Ben simply refuses to say ITS CLOBBERIN TIME Can t wait to see this in IMAX


In [8]:
# vectorization
# convert text to vector for SVM
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preprocessed_text) 
y = np.asarray(data["sentiment"]) # label
# print(vectorizer.get_feature_names_out())
# print(X)
print(y)

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


In [9]:
# split train data and test data and train SVM model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = SVC(kernel="linear").fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(X_train.shape)
print(X_train)


              precision    recall  f1-score   support

          -1       0.64      0.93      0.76        15
           0       0.00      0.00      0.00         5
           1       0.71      0.56      0.62         9

    accuracy                           0.66        29
   macro avg       0.45      0.50      0.46        29
weighted avg       0.55      0.66      0.59        29

(112, 893)
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2070 stored elements and shape (112, 893)>
  Coords	Values
  (0, 736)	0.6813709773976496
  (0, 826)	0.7319382427228214
  (1, 68)	0.6265211198953019
  (1, 211)	0.2976267329180903
  (1, 219)	0.3726969936719881
  (1, 262)	0.3726969936719881
  (1, 471)	0.2844085205065316
  (1, 514)	0.31326055994765095
  (1, 778)	0.15015741086187584
  (1, 790)	0.19878602315352212
  (2, 45)	0.6106818484156334
  (2, 422)	0.2439585311310289
  (2, 472)	0.6420342450435588
  (2, 512)	0.256937283837657
  (2, 838)	0.29887652211160864
  (3, 50)	0.12688710574260692
  (3, 

In [10]:
# save the model
import joblib

# store the trained model
joblib.dump(model, "../model/svm_sentiment_model.pkl")

# Save the vectorizer (important for preprocessing new text consistently)
# joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


['../model/svm_sentiment_model.pkl']

In [11]:
#load the trained model
load_model = joblib.load("../model/svm_sentiment_model.pkl")
# Predict sentiment and test model
new_review = "I do not want to pay for that, the film is stupid"
new_processed = preprocess_data(new_review)
X_new = vectorizer.transform([new_processed])
result = load_model.predict(X_new) # -1 = negative, 1 = positive, 0 = neutral
if result == 1:
    print("Positive")
elif result == 0:
    print("Neural")
else:
    print("Negative")

Negative
